In [49]:
import numpy as np
import pandas as pd
import displacement_solver
import constitutive
import mesh_gen
import quadrature
import stress_gauss
import patch_n_int_nodes
import gauss_pt_coord
import stress_nodes_dc
import spr_stress

from sklearn.preprocessing import MinMaxScaler 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
from tensorflow.keras.models import model_from_json

In [50]:
# ............................Inputs Parameters......................#

#Doamin geometry
domain_coord = np.array([[2, 0], [20, 0], [20, 10], [10, 0], [0, 2]]);

# Body force components
b = np.array([[0], [0]])

# Traction Components
q = 1/8      #unirt force for unit length

T = np.array([[q, 0]]);

# Young's modulus
E = 1.0

# Poission's ration
nu = 1/3;

# problem type (0--->plane stress, 1---->plane strain)
problem_type = 0;

#Element type used for meshing (0---->4 nodes quadrilateral)
el_type = 0;

# No. of Gauss points required forintegration
ngp2d = 1;
ngp1d = 2;

# Number of mesh in one direction.
N = 5;

In [51]:
u = displacement_solver.solve_fem_plat_with_hole(N, E, nu, ngp2d, ngp1d, el_type, problem_type, domain_coord, b, T)
# print(u_node)

In [182]:
# ms = mesh size ----> Number of element in one direction
ms = N;

#total number of elements
nel = 2*ms*ms 

# reshaping u into u_nodes with displacement in x-direction in first column and y-direciton in second column
u_nodes = u.reshape((2*(ms+1)*(ms+1)-(ms+1), 2))
# print(u_nodes)

In [142]:
# constitutive relation matrix, calculated using the fuction "Constitutube" with input E->(Young's Modulus), nu(Poisson's raton), problem_type(plane stress or plane strain)
C = constitutive.constitutive(E, nu, problem_type)
# print(C);

In [143]:
# mesh generation 
nx = ms #number of element in x-direction
ny = ms #number of element in y-direction

# el_type = element type specifiedc. 0----> q4, q-----> q8, 2-----> q9
el_type = 0;

#mesh_obj = object created to calculated nodal coordinates ans connectivity array using functon "connectivity" and "coord_array" 
#input nx->number of element in x-direction, xy->number of element in y-direction, domain_coord->coordinates of the corner points and mid-points of the cook's skew beam problem, el_type->element type specidied.

mesh_obj = mesh_gen.platWithHole(nx, ny, domain_coord.reshape(10, 1), el_type);
connect = mesh_obj.connectivity();
# print(connect)
coord = mesh_obj.coord_array();

In [145]:
strs_ref = pd.read_csv('Data/superconv_gauss_stress_ms_256.csv')
# strs_ref = strs_ref.abs()

In [146]:
# print(strs_ref)

In [155]:
X = pd.read_csv('Coord/coord_gauss_pt_256.csv')
X = X.loc[:, ~X.columns.str.contains('Unnamed')];
y = strs_ref;
y = y.loc[:, ~y.columns.str.contains('Unnamed')];
# y = y.loc[:, ~y.columns.str.contains('x')];
# y = y.drop(columns=['x', 'y'])
# y = y.loc[:, ~y.columns.str.contains('y')];

In [156]:
# print(X.head())
# print(y.head())

In [157]:
scaler = MinMaxScaler()
scaler.fit(X)
scaled_x = scaler.transform(X)
scaled_test = scaler.transform(coord)

/home/krishna/Desktop/btp_499_code/btp_env/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [158]:
# print(X.head());
# print(scaled_x);
# print(scaled_test);
# print(len(scaled_x));
# print(len(scaled_test));

In [159]:
#ANN one model in whole domain for 3 output layer

model1 = Sequential()
model1.add(Dense(units=50,  activation='relu'))
model1.add(Dense(units=40, activation='relu'))
model1.add(Dense(units=30, activation='relu'))
model1.add(Dense(units=20, activation='relu'))
model1.add(Dense(units=10, activation='relu'))
model1.add(Dense(units=3, activation='relu'))
model1.compile(
    loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))



In [160]:
 es = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',  mode = 'min', verbose = 1, patience = 50);
 np.random.seed(1)
 history = model1.fit(x = scaled_x, y = y, batch_size=256, epochs=300, verbose = 1, validation_split = 0.1, callbacks= [es]);

Epoch 1/300
461/461 [==============================] - 14s 15ms/step - loss: 710294200989515776.0000 - val_loss: 1793456078848.0000
Epoch 2/300
461/461 [==============================] - 6s 13ms/step - loss: 710293788672655360.0000 - val_loss: 1793456078848.0000
Epoch 3/300
461/461 [==============================] - 6s 13ms/step - loss: 710294132270039040.0000 - val_loss: 1793456078848.0000
Epoch 4/300
461/461 [==============================] - 6s 13ms/step - loss: 710294132270039040.0000 - val_loss: 1793456078848.0000
Epoch 5/300
461/461 [==============================] - 6s 13ms/step - loss: 710294200989515776.0000 - val_loss: 1793456078848.0000
Epoch 6/300
461/461 [==============================] - 6s 13ms/step - loss: 710293857392132096.0000 - val_loss: 1793456078848.0000
Epoch 7/300
461/461 [==============================] - 6s 13ms/step - loss: 710294200989515776.0000 - val_loss: 1793456078848.0000
Epoch 8/300
461/461 [==============================] - 6s 13ms/step - loss: 710294

In [169]:
var = 'x_y_xy'

In [170]:
loss = pd.DataFrame(model1.history.history);

In [163]:
# import matplotlib.pyplot as plt
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('train_loss', fontsize=15)
# plt.ylabel('Loss', fontsize=15)
# plt.xlabel('Epoch', fontsize=15)
# plt.legend(['train', 'val'], loc='upper right')
# plt.xticks(fontsize=15)
# plt.yticks(fontsize=15)
# plt.show()

In [171]:
#Saving train model with different optimizer.
model_json = model1.to_json()
with open('Model/model1_'+var+'.json', 'w') as json_file:
    json_file.write(model_json)

#serialize weights to HDF5 file
model1.save_weights('Model/model1_'+var+'.h5')


In [172]:
# # load json and create model
json_file = open('Model/model1_x_y_xy.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json);

# #load weights into new model
loaded_model.load_weights('Model/model1_x_y_xy.h5') 

In [173]:
#Stress calculation at Gauss points
stress = np.zeros((nel, ngp2d*ngp2d, 3))
strains = np.zeros((nel, ngp2d*ngp2d, 3))

for i in range(nel):
    stress_i_g = np.zeros((ngp2d*ngp2d, 3))
    strains_i_g = np.zeros((ngp2d*ngp2d, 3))

    stress_i_g, strains_i_g = stress_gauss.get_element_stress(
        i, ngp2d, el_type, connect, coord, u, C)

    stress[i][:][:] = stress_i_g
    strains[i][:][:] = strains_i_g.reshape((1, 3))


In [174]:
# calcuation of gauss coordinates.
gauss_coords = np.zeros((nel, ngp2d*ngp2d, 2))
gp, weights = quadrature.quadrature(ngp2d)
for i in range(nel):
    node = connect[i, :]
    vertex_coord = coord[node, :].reshape(-1)
    gauss_coords[i][:][:] = gauss_pt_coord.gauss_pts(
        ngp2d, vertex_coord, gp, el_type)
gauss_coords = gauss_coords.reshape(gauss_coords.shape[0], -1)


In [175]:
#creation of patches for spr_stress;
patch, n_patches, int_nodes = patch_n_int_nodes.patch_n_int_nodes_plat_with_hole(ms)


In [205]:
#spr_stress STRESS Calculations
stress_spr = spr_stress.spr_plat_with_hole(gauss_coords, coord, connect, stress,
                     int_nodes, n_patches, patch, ms)
# stress_spr


In [206]:
#Directly calculated stress
stress_dc, strain_dc = stress_nodes_dc.stress_dc(
    connect, coord, u, nel, el_type, C);


In [207]:
#outter points in
sp = []
# print(ms)
# ms = 4
temp = ms+1
for i in range(2*ms-1):
    sp.append(temp)
    sp.append(temp+ms)
    temp = temp+ms+1
for i in range(ms+1):
    sp.append(temp)
    temp = temp+1

for i in range(ms+1):
    sp.append(i)

sp = sorted(sp)
print(sp);

[0, 1, 2, 3, 4, 5, 6, 11, 12, 17, 18, 23, 24, 29, 30, 35, 36, 41, 42, 47, 48, 53, 54, 59, 60, 61, 62, 63, 64, 65]


In [208]:
coord_corner = coord[sp]
# print(coord_corner)

In [209]:
scaled_test_corner = scaler.transform(coord_corner)
# scaled_test_corner

/home/krishna/Desktop/btp_499_code/btp_env/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [211]:
ref_stress = pd.read_csv('ref_stress/ref_stress_for_ms_5_from_320.csv');
ref_stress = ref_stress.loc[:, ~ref_stress.columns.str.contains('^Unnamed')];
ref_stress = ref_stress.drop(columns=['x', 'y'])
# print(ref_stress)

ref_stress = ref_stress.to_numpy();
# ref_stress

In [212]:
a = ref_stress.T
# print(len(a));

In [213]:
b = stress_spr.T
# print(len(b))

In [218]:
err_spr_abs = np.abs((a-b));
err_spr_perc = np.abs((a-b)/a);
err_dc_abs = np.abs((a-stress_dc.T));
err_dc_perc = np.abs((a-stress_dc.T)/a);
# err_spr_abs
# err_dc_abs

In [223]:
test = pd.DataFrame(scaled_test, columns =['a', 'b']);
# test

In [227]:
c = loaded_model.predict(test);
# len(c)

3/3 [==============================] - 0s 2ms/step


In [229]:
err_ann_abs = np.abs((a-c.T))
err_ann_perc = np.abs((a-c.T)/a)
# err_ann_abs

In [230]:

alll = np.concatenate([coord, 
                        a.reshape(a.T.shape[0], 3), 
                        b.reshape(b.T.shape[0], 3), 
                        c.reshape(c.shape[0], 3), 
                        err_spr_abs.reshape(err_spr_abs.T.shape[0], 3), 
                        err_ann_abs.reshape(err_ann_abs.T.shape[0],3), 
                        err_dc_abs.reshape(err_dc_abs.T.shape[0],3),
                        err_spr_perc.reshape(err_spr_perc.T.shape[0],3),
                        err_ann_perc.reshape(err_ann_perc.T.shape[0],3),
                        err_dc_perc.reshape(err_dc_perc.T.shape[0],3) 
                        ], 
                        axis = 1);
# alll

In [239]:
tdf =pd.DataFrame(alll, columns=['x', 
                                'y', 
                                'ref_stress_x',
                                'ref_stress_y',
                                'ref_stress_xy', 
                                'spr_stress_x',
                                'spr_stress_y',
                                'spr_stress_xy', 
                                'ann_stress_x',
                                'ann_stress_y',
                                'ann_stress_xy', 
                                'err_spr_abs_x',
                                'err_spr_abs_y',
                                'err_spr_abs_xy', 
                                'err_ann_abs_x',
                                'err_ann_abs_y',
                                'err_ann_abs_xy', 
                                'err_dc_abs_x',
                                'err_dc_abs_y',
                                'err_dc_abs_xy',
                                'err_spr_perc_x',
                                'err_spr_perc_y',
                                'err_spr_perc_xy', 
                                'err_ann_perc_x',
                                'err_ann_perc_y',
                                'err_ann_perc_xy', 
                                'err_dc_perc_x',
                                'err_dc_perc_y',
                                'err_dc_perc_xy'
                                ]);
print(tdf);

       x    y  ref_stress_x   ref_stress_y  ref_stress_xy  spr_stress_x  \
0    2.0  0.0 -1.726256e-04       0.144666      -9.529965     -0.977762   
1    5.6  0.0  7.839685e+00       2.178640       2.840977     -0.072839   
2    9.2  0.0 -7.085356e-05     -91.354094    -251.462422     -0.687458   
3   12.8  0.0  1.315752e+01       5.933357       1.333309     -0.303101   
4   16.4  0.0  9.285278e-04   -3946.754606  -10292.325574     -0.482481   
..   ...  ...           ...            ...            ...           ...   
61   2.0  1.6  1.207845e+06   33628.968404      20.312187     -0.663953   
62   4.0  1.2  9.081374e-02 -200224.363378 -285082.957893      0.631973   
63   6.0  0.8  1.029317e+04   -6926.363241      -1.424634     -0.369132   
64   8.0  0.4 -2.817225e-02     229.821983     -52.651556      0.940496   
65  10.0  0.0 -1.167300e+02      57.765924     -25.935194     -0.095676   

    spr_stress_y  spr_stress_xy  ann_stress_x  ann_stress_y  ...  \
0      -0.694206      -0.216256

In [236]:
tdf.to_csv('Results/all_matlab_train_model1_ref_'+var+'.csv');
tdf.head()

,x,y,ref_stress_x,ref_stress_y,ref_stress_xy,spr_stress_x,spr_stress_y,spr_stress_xy,ann_stress_x,ann_stress_y,...,err_dc_abs_xy,err_spr_perc_x,err_spr_perc_y,err_spr_perc_xy,err_ann_perc_x,err_ann_perc_y,err_ann_perc_xy,err_dc_perc_x,err_dc_perc_y,err_dc_perc_xy
0,2.0,0.0,-0.000173,0.144666,-9.529965,-0.977762,-0.694206,-0.216256,0.0,0.0,...,10.428038,5663.061093,5.798695,0.977308,1.0,1.0,1.0,61341.118638,27.907634,1.094237
1,5.6,0.0,7.839685,2.178640,2.840977,-0.072839,-0.037187,0.069772,0.0,0.0,...,3.660670,1.009291,1.017069,0.975441,1.0,1.0,1.0,1.164220,0.844558,1.288525
2,9.2,0.0,-0.000071,-91.354094,-251.462422,-0.687458,-1.050563,-0.762537,0.0,0.0,...,249.095729,9701.518261,0.988500,0.996968,1.0,1.0,1.0,173905.116993,1.021424,0.990588
3,12.8,0.0,13.157520,5.933357,1.333309,-0.303101,-0.069180,0.015766,0.0,0.0,...,0.261244,1.023036,1.011660,0.988175,1.0,1.0,1.0,0.955942,1.091159,0.195937
4,16.4,0.0,0.000929,-3946.754606,-10292.325574,-0.482481,-1.214000,-0.727692,0.0,0.0,...,10293.383032,520.619879,0.999692,0.999929,1.0,1.0,1.0,9889.297830,0.998992,1.000103
